In [1]:
!pip install torch torchvision

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
from typing import List, Dict
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader

In [4]:
#########################################    EMBED DATA   ##################################################

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import re

def parse_trajectories(file_path):
    with open(file_path, 'r') as f:
        content = f.read()

    blocks = content.strip().split("\n\n")  # Split by blocks separated by blank lines
    all_trajectories = []

    for block in blocks:
        lines = block.strip().split("\n")
        trajectory = []

        for line in lines:
            actions = line.strip().split("-")  # Each action is separated by '-'
            timestep = []

            for action in actions:
                # Adjust the regex to capture multiple labels in the format pX(y1,y2,...)
                match = re.match(r"r(\d+)\((\d+)\):p(\d+)\(([^)]+)\),p(\d+)\(([^)]+)\)", action.strip())
                if match:
                    r_id, duration, p_start, y_start_str, p_end, y_end_str = match.groups()
                    
                    # Extract numeric label values (e.g., 'y1', 'y7' → 1, 7)
                    y_start = [int(label[1:]) for label in y_start_str.split(',')]
                    y_end = [int(label[1:]) for label in y_end_str.split(',')]

                    timestep.append({
                        "robot_id": int(r_id),
                        "duration": int(duration),
                        "p_start": int(p_start),
                        "y_start": y_start,
                        "p_end": int(p_end),
                        "y_end": y_end
                    })
                else:
                    print(f"Warning: Could not parse action: {action}")
            trajectory.append(timestep)

        all_trajectories.append(trajectory)

    return all_trajectories

# Parse the data from the log file
parsed_data = parse_trajectories("log.txt")

# Print the parsed data for the first timestep in the first trajectory
print(parsed_data[0][0])  # first timestep in that trajectory

[{'robot_id': 1, 'duration': 8, 'p_start': 2, 'y_start': [1, 7], 'p_end': 14, 'y_end': [4]}, {'robot_id': 4, 'duration': 10, 'p_start': 2, 'y_start': [1], 'p_end': 1, 'y_end': [2]}]


In [6]:
import torch
import torch.nn as nn

# Compute max ids from the parsed data
robot_ids = set()
place_ids = set()

for traj in parsed_data:
    for timestep in traj:
        for action in timestep:
            robot_ids.add(action['robot_id'])
            place_ids.add(action['p_start'])
            place_ids.add(action['p_end'])

max_robot_id = max(robot_ids) + 1
max_place_id = max(place_ids) + 1
max_timesteps = max(len(traj) for traj in parsed_data)

# Then initialize your embeddings
embed_dim = 32
robot_embedding_matrix = nn.Embedding(max_robot_id, embed_dim)
place_embedding_matrix = nn.Embedding(max_place_id, embed_dim)
timestep_embedding_matrix = nn.Embedding(max_timesteps, embed_dim)

# If you want to just use tensors here, extract their weight data:
robot_embeddings = robot_embedding_matrix.weight.data  # shape (max_robot_id, embed_dim)
place_embeddings = place_embedding_matrix.weight.data  # shape (max_place_id, embed_dim)


In [7]:
def sinusoidal_embedding(value, dim=32):
    """Sinusoidal embedding for a scalar value."""
    pe = torch.zeros(dim)
    position = torch.tensor(value, dtype=torch.float32)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(np.log(10000.0) / dim))
    pe[0::2] = torch.sin(position * div_term)
    pe[1::2] = torch.cos(position * div_term)
    return pe

def time_embedding(duration, max_duration):
    """Sinusoidal encoding for duration using 2D angle."""
    normalized = np.log(duration + 1) / np.log(max_duration + 1)
    angle = 2 * np.pi * normalized
    return torch.tensor([np.sin(angle), np.cos(angle)], dtype=torch.float32)

def label_embedding(y_start, y_end, num_labels=5):
    """Label transition encoding: -1 for origin labels, 1 for destination labels, 0 otherwise."""
    vec = torch.zeros(num_labels)
    for i in y_start:
        vec[i - 1] = -1
    for i in y_end:
        if vec[i - 1] == -1:
            vec[i - 1] = 0  # neutralize if label is both in start and end
        else:
            vec[i - 1] = 1
    return vec

def timestep_embedding(position, dim=32):
    """Sinusoidal positional encoding for timestep index."""
    pe = torch.zeros(dim)
    position = torch.tensor(position, dtype=torch.float32)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(np.log(10000.0) / dim))
    pe[0::2] = torch.sin(position * div_term)
    pe[1::2] = torch.cos(position * div_term)
    return pe

In [13]:
def encode_actions(parsed_data, robot_embeddings, place_embeddings, timestep_embeddings, num_labels=5, embed_dim=32):
    max_duration = max(
        action['duration']
        for trajectory in parsed_data
        for timestep in trajectory
        for action in timestep
    )
    
    encoded_vectors = []

    for trajectory in parsed_data:
        encoded_trajectory = []

        for t_idx, timestep in enumerate(trajectory):
            timestep_vec = timestep_embeddings[t_idx]

            encoded_timestep = []

            for action in timestep:
                # Instead of sinusoidal, do lookup here:
                robot_vec = robot_embeddings[action['robot_id']]  
                p_start_vec = place_embeddings[action['p_start']]
                p_end_vec = place_embeddings[action['p_end']]

                label_vec = label_embedding(action['y_start'], action['y_end'], num_labels=num_labels)
                time_vec = time_embedding(action['duration'], 10)

                full_vec = torch.cat([
                    robot_vec,
                    p_start_vec,
                    p_end_vec,
                    label_vec,
                    time_vec,
                    timestep_vec
                ])

                encoded_timestep.append(full_vec)
            encoded_trajectory.append(encoded_timestep)

        encoded_vectors.append(encoded_trajectory)

    return encoded_vectors


In [ ]:
# Parse the data from the log file
parsed_data = parse_trajectories("Datasets/Dataset1/DataSet_Balanced.txt")

# Set the number of labels manually
num_labels = 7  # Change this based on the number of labels you want

# Example: create embedding matrices (trainable parameters should be managed separately during training)
robot_embedding_matrix = nn.Embedding(max_robot_id, embed_dim)
place_embedding_matrix = nn.Embedding(max_place_id, embed_dim)

# Pass the weight tensors:
encoded_vectors = encode_actions(
    parsed_data,
    robot_embedding_matrix.weight.data,
    place_embedding_matrix.weight.data,
    timestep_embeddings=timestep_embedding_matrix.weight.data,
    num_labels=num_labels,
    embed_dim=embed_dim
)

# Print the encoded vector for the first timestep in the first trajectory
print(encoded_vectors[0][0][0])  # First timestep in the first trajectory
print(f"Vector size: {encoded_vectors[0][0][0].shape}")  # Expected size

In [14]:
#########################################   PREPARE DATA    #######################################################

In [15]:
import torch

def pad_trajectories(encoded_vectors):
    max_timesteps = max(len(traj) for traj in encoded_vectors)
    max_actions = max(len(ts) for traj in encoded_vectors for ts in traj)
    feature_dim = encoded_vectors[0][0][0].shape[-1]

    padded_trajectories = []
    action_masks = []
    timestep_masks = []

    for traj in encoded_vectors:
        padded_traj = []
        traj_action_mask = []
        for ts in traj:
            # Pad actions within this timestep
            real_actions = len(ts)
            padded_ts = ts + [torch.zeros(feature_dim)] * (max_actions - real_actions)
            padded_ts_tensor = torch.stack(padded_ts)  # (A, D)
            padded_traj.append(padded_ts_tensor)

            # Action mask: 1 for real actions, 0 for padded
            action_mask = [1] * real_actions + [0] * (max_actions - real_actions)
            traj_action_mask.append(torch.tensor(action_mask, dtype=torch.bool))  # (A,)

        # Pad timesteps
        num_real_timesteps = len(traj)
        empty_ts = torch.zeros(max_actions, feature_dim)
        padded_traj += [empty_ts] * (max_timesteps - num_real_timesteps)
        padded_traj_tensor = torch.stack(padded_traj)  # (T, A, D)
        padded_trajectories.append(padded_traj_tensor)

        # Pad action mask
        empty_action_mask = torch.zeros(max_actions, dtype=torch.bool)
        traj_action_mask += [empty_action_mask] * (max_timesteps - num_real_timesteps)
        action_masks.append(torch.stack(traj_action_mask))  # (T, A)

        # Timestep mask: 1 for real timesteps, 0 for padded
        timestep_mask = [1] * num_real_timesteps + [0] * (max_timesteps - num_real_timesteps)
        timestep_masks.append(torch.tensor(timestep_mask, dtype=torch.bool))  # (T,)

    return (
        torch.stack(padded_trajectories),  # (B, T, A, D)
        torch.stack(action_masks),         # (B, T, A)
        torch.stack(timestep_masks),       # (B, T)
    )


data_tensor, action_mask, timestep_mask = pad_trajectories(encoded_vectors)

In [16]:
print("Data Tensor Shape:", data_tensor.shape)          # (B, T, A, D)
print("Action Mask Shape:", action_mask.shape)          # (B, T, A)
print("Timestep Mask Shape:", timestep_mask.shape)      # (B, T)

# --- Check a specific trajectory ---
i = 0  # Check the first trajectory

print(f"\nTrajectory {i}:")
print("Timestep lengths:", [sum(am).item() for am in action_mask[i]])  # Number of real actions per timestep
print("Timestep mask:", timestep_mask[i].int().tolist())               # 1 if real timestep, 0 if padded

# --- Visual check for one timestep ---
t = 2  # First timestep
print(f"\nTrajectory {i}, Timestep {t} — Action Mask:", action_mask[i][t].int().tolist())
print(f"Data tensor (values) shape: {data_tensor[i][t].shape}")
print("First action vector (real or zero):", data_tensor[i][t][0])  # Show first 5 dims of first action

Data Tensor Shape: torch.Size([5200, 39, 4, 137])
Action Mask Shape: torch.Size([5200, 39, 4])
Timestep Mask Shape: torch.Size([5200, 39])

Trajectory 0:
Timestep lengths: [1, 4, 2, 2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Timestep mask: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Trajectory 0, Timestep 2 — Action Mask: [1, 1, 0, 0]
Data tensor (values) shape: torch.Size([4, 137])
First action vector (real or zero): tensor([ 0.9777, -0.7392, -0.4093, -0.2958,  0.9308, -0.8573, -0.0022, -0.5086,
        -0.3886, -0.4624, -1.5779,  0.2407, -0.4397, -0.6403, -1.3094,  0.9380,
        -1.1098,  0.8411,  0.0365, -1.6489,  0.5481,  0.8254, -0.8321, -0.8304,
        -0.2717, -0.0426, -0.1333, -1.7681, -0.8077,  1.2070,  0.4288,  1.7007,
        -0.8839, -0.7609,  1.3325,  0.3432,  1.2239, -0.4532,  1.0612,  0.6864,
        -1.3334, -0.6102,  0.3203, -0.

In [66]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.data import TensorDataset, Subset

# Create full dataset
dataset = TensorDataset(data_tensor, action_mask, timestep_mask)

normal_indices = list(range(0, 1000)) + list(range(1100, 5100))   # 5000 normal
abnormal_indices = list(range(1000, 1100)) + list(range(5100, 5200))  # 200 abnormal

# Split normal indices into 80/10/10
train_len = int(0.8 * len(normal_indices))
val_len = int(0.1 * len(normal_indices))
test_normal_len = len(normal_indices) - train_len - val_len

# Shuffle deterministically
g = torch.Generator().manual_seed(30)
permuted = torch.randperm(len(normal_indices), generator=g).tolist()

train_indices = [normal_indices[i] for i in permuted[:train_len]]
val_indices   = [normal_indices[i] for i in permuted[train_len:train_len + val_len]]
test_indices  = [normal_indices[i] for i in permuted[train_len + val_len:]]

# Add abnormal indices to test set
test_indices += abnormal_indices

train_dataset = Subset(dataset, train_indices)
val_dataset   = Subset(dataset, val_indices)
test_dataset  = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16)
test_loader  = DataLoader(test_dataset, batch_size=16)


In [68]:
print("Train batches:", len(train_loader))
print("Validation batches:", len(val_loader))
print("Test batches:", len(test_loader))

# Check one batch
for data_batch, action_mask_batch, timestep_mask_batch in train_loader:
    print("Data batch shape:", data_batch.shape)  # (B, T, A, D)
    print("Action mask shape:", action_mask_batch.shape)  # (B, T, A)
    print("Timestep mask shape:", timestep_mask_batch.shape)  # (B, T)
    break


Train batches: 250
Validation batches: 32
Test batches: 44
Data batch shape: torch.Size([16, 39, 4, 137])
Action mask shape: torch.Size([16, 39, 4])
Timestep mask shape: torch.Size([16, 39])


In [70]:
###################################################    MODEL ARCHITECTURE    #########################################

In [72]:
import torch
import torch.nn as nn

class ActionTransformerEncoder(nn.Module):
    def __init__(self, input_dim, embed_dim, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
    def forward(self, x, mask):
        # x: (B*T, A, embed_dim)
        # mask: (B*T, A) bool mask, True=valid
        key_padding_mask = ~mask  # Transformer expects True for padding tokens
        out = self.transformer_encoder(x, src_key_padding_mask=key_padding_mask)
        return out  # (B*T, A, embed_dim)

class TrajectoryAutoencoder(nn.Module):
    def __init__(self, input_dim, embed_dim=64, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, embed_dim)
        
        self.action_encoder = ActionTransformerEncoder(
            input_dim=input_dim,
            embed_dim=embed_dim,
            nhead=nhead,
            num_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        
        timestep_encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.timestep_encoder = nn.TransformerEncoder(
            timestep_encoder_layer,
            num_layers=num_layers
        )
        
        # Decoder projects from embed_dim back to original feature dim (D)
        self.decoder = nn.Linear(embed_dim, input_dim)
        
    def forward(self, x, action_mask, timestep_mask):
        # x: (B, T, A, D)
        B, T, A, D = x.shape
        
        # Flatten timesteps to batch dim for action encoder
        x_flat = x.view(B * T, A, D)  # (B*T, A, D)
        action_mask_flat = action_mask.view(B * T, A)  # (B*T, A)
        
        # Input projection
        x_proj = self.input_proj(x_flat)  # (B*T, A, embed_dim)
        
        # Encode actions
        action_encoded = self.action_encoder(x_proj, action_mask_flat)  # (B*T, A, embed_dim)
        
        # Reshape back to (B, T, A, embed_dim)
        action_encoded = action_encoded.view(B, T, A, -1)
        
        # Aggregate per timestep: mean over valid actions (mask-aware mean)
        # Masked mean: sum / count
        masked_sum = (action_encoded * action_mask.unsqueeze(-1)).sum(dim=2)  # (B, T, embed_dim)
        valid_counts = action_mask.sum(dim=2).clamp(min=1).unsqueeze(-1)  # avoid div by 0
        timestep_repr = masked_sum / valid_counts  # (B, T, embed_dim)
        
        # Encode timesteps
        # Transformer expects key_padding_mask True for padded
        key_padding_mask = ~timestep_mask  # (B, T)
        timestep_encoded = self.timestep_encoder(timestep_repr, src_key_padding_mask=key_padding_mask)  # (B, T, embed_dim)
        
        # Decode per action: replicate timestep_encoded per action
        timestep_expanded = timestep_encoded.unsqueeze(2).repeat(1, 1, A, 1)  # (B, T, A, embed_dim)
        
        out = self.decoder(timestep_expanded)  # (B, T, A, D)
        
        return out



In [74]:
###########################################    TRAIN    #########################################################

In [76]:
import torch
import torch.nn.functional as F

def masked_mse_loss(pred, target, mask):
    # pred, target: (B, T, A, D)
    # mask: (B, T, A) bool, True=valid data points
    mask = mask.unsqueeze(-1)  # (B, T, A, 1)
    diff = (pred - target) ** 2
    diff = diff * mask  # zero out padded
    return diff.sum() / mask.sum().clamp(min=1)  # mean over valid elements

def train_model(model, train_loader, val_loader, optimizer, device, epochs=20):
    model.to(device)
    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for x, action_mask, timestep_mask in train_loader:
            x = x.to(device)
            action_mask = action_mask.to(device)
            timestep_mask = timestep_mask.to(device)

            optimizer.zero_grad()
            out = model(x, action_mask, timestep_mask)

            # Combine masks for valid actions inside valid timesteps
            combined_mask = action_mask & timestep_mask.unsqueeze(-1)

            loss = masked_mse_loss(out, x, combined_mask)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * x.size(0)

        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, action_mask, timestep_mask in val_loader:
                x = x.to(device)
                action_mask = action_mask.to(device)
                timestep_mask = timestep_mask.to(device)

                out = model(x, action_mask, timestep_mask)
                combined_mask = action_mask & timestep_mask.unsqueeze(-1)
                loss = masked_mse_loss(out, x, combined_mask)
                val_loss += loss.item() * x.size(0)

        val_loss /= len(val_loader.dataset)

        print(f"Epoch {epoch+1}/{epochs} — Train Loss: {train_loss:.4f} — Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_trajectory_autoencoder.pth")

# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TrajectoryAutoencoder(input_dim=data_tensor.shape[-1], embed_dim=64)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_model(model, train_loader, val_loader, optimizer, device, epochs=30)


Epoch 1/30 — Train Loss: 57.1254 — Val Loss: 42.9894
Epoch 2/30 — Train Loss: 42.6846 — Val Loss: 41.7647
Epoch 3/30 — Train Loss: 41.8787 — Val Loss: 41.5219
Epoch 4/30 — Train Loss: 41.6754 — Val Loss: 41.4052
Epoch 5/30 — Train Loss: 41.5510 — Val Loss: 41.3487
Epoch 6/30 — Train Loss: 41.4630 — Val Loss: 41.3222
Epoch 7/30 — Train Loss: 41.4330 — Val Loss: 41.2905
Epoch 8/30 — Train Loss: 41.4094 — Val Loss: 41.2826
Epoch 9/30 — Train Loss: 41.4152 — Val Loss: 41.2708
Epoch 10/30 — Train Loss: 41.4059 — Val Loss: 41.2637
Epoch 11/30 — Train Loss: 41.3806 — Val Loss: 41.2621
Epoch 12/30 — Train Loss: 41.3672 — Val Loss: 41.2556
Epoch 13/30 — Train Loss: 41.3659 — Val Loss: 41.2557
Epoch 14/30 — Train Loss: 41.3594 — Val Loss: 41.2527
Epoch 15/30 — Train Loss: 41.3813 — Val Loss: 41.2498
Epoch 16/30 — Train Loss: 41.3583 — Val Loss: 41.2491
Epoch 17/30 — Train Loss: 41.3727 — Val Loss: 41.2509
Epoch 18/30 — Train Loss: 41.3581 — Val Loss: 41.2517
Epoch 19/30 — Train Loss: 41.3420 — V

In [77]:
###########################################    VISUALIZE RESULTS    #########################################################

In [84]:
def evaluate_anomalies(model, test_loader, device, test_indices, top_k=5):
    """
    Evaluates anomalies by computing reconstruction loss for each trajectory and
    associating it with the original dataset index using `test_indices`.

    Args:
        model (torch.nn.Module): Trained model.
        test_loader (DataLoader): DataLoader for the test set.
        device (torch.device): Device to run inference on.
        test_indices (list of int): Original indices corresponding to test_loader data.
        top_k (int): Number of top anomalies to display.

    Returns:
        sorted_pairs (list of tuples): List of (original_index, loss), sorted by loss descending.
    """
    model.eval()
    losses = []
    ids = []

    with torch.no_grad():
        start = 0  # Pointer in test_indices
        for x, action_mask, timestep_mask in test_loader:
            x = x.to(device)
            action_mask = action_mask.to(device)
            timestep_mask = timestep_mask.to(device)

            out = model(x, action_mask, timestep_mask)

            # Calculate per-sample loss
            mask = action_mask & timestep_mask.unsqueeze(-1)
            diff = (out - x) ** 2 * mask.unsqueeze(-1)
            loss_per_sample = diff.sum(dim=[1, 2, 3]) / mask.sum(dim=[1, 2]).clamp(min=1)

            # Map to original IDs
            batch_size = x.size(0)
            batch_ids = test_indices[start:start + batch_size]
            start += batch_size

            losses.extend(loss_per_sample.cpu().tolist())
            ids.extend(batch_ids)

    # Pair (original ID, loss) and sort
    sorted_pairs = sorted(zip(ids, losses), key=lambda x: x[1], reverse=True)

    print(f"Top {top_k} most abnormal trajectories (ID and reconstruction loss):")
    for traj_id, loss in sorted_pairs[:top_k]:
        print(f"Trajectory ID {traj_id}: Loss = {loss:.4f}")

    return sorted_pairs


In [88]:
def evaluate_precision_at_k(sorted_pairs, top_k=50, abnormal_ranges=None):
    """
    Calculates precision@k for detecting abnormal trajectories.

    Args:
        sorted_pairs (list of tuples): (trajectory_id, loss) pairs sorted by loss descending.
        top_k (int): Number of top trajectories to consider.
        abnormal_ranges (list of ints): List of trajectory IDs considered abnormal.

    Returns:
        abnormal_detected (list of tuples): Abnormal (ID, loss) found in top_k.
        precision (float): Precision@k value.
    """
    if abnormal_ranges is None:
        abnormal_ranges = list(range(1000, 1100)) + list(range(5100, 5200))

    top_k_subset = sorted_pairs[:top_k]
    abnormal_detected = [(traj_id, loss) for traj_id, loss in top_k_subset if traj_id in abnormal_ranges]

    print(f"\nDetected {len(abnormal_detected)} truly abnormal trajectories in top {top_k}:")
    for traj_id, loss in abnormal_detected:
        print(f"Trajectory ID {traj_id}: Loss = {loss:.4f}")

    precision = len(abnormal_detected) / top_k
    print(f"\nPrecision@{top_k} = {precision:.2f}")

    return abnormal_detected, precision

sorted_pairs = evaluate_anomalies(model, test_loader, device, test_indices=test_indices, top_k=500)

abnormal_detected, precision = evaluate_precision_at_k(sorted_pairs, top_k=100)


Top 500 most abnormal trajectories (ID and reconstruction loss):
Trajectory ID 1022: Loss = 61.1130
Trajectory ID 1017: Loss = 60.2809
Trajectory ID 1056: Loss = 59.8945
Trajectory ID 5108: Loss = 59.4831
Trajectory ID 5103: Loss = 59.4693
Trajectory ID 5142: Loss = 59.3265
Trajectory ID 1058: Loss = 58.8943
Trajectory ID 1093: Loss = 58.7785
Trajectory ID 1010: Loss = 58.2182
Trajectory ID 5133: Loss = 58.0130
Trajectory ID 1062: Loss = 58.0024
Trajectory ID 5179: Loss = 57.9569
Trajectory ID 1019: Loss = 57.7927
Trajectory ID 1031: Loss = 57.7583
Trajectory ID 1081: Loss = 57.6966
Trajectory ID 1024: Loss = 57.3848
Trajectory ID 1097: Loss = 57.3369
Trajectory ID 5181: Loss = 56.7913
Trajectory ID 1046: Loss = 56.6227
Trajectory ID 5127: Loss = 56.3987
Trajectory ID 1086: Loss = 56.3360
Trajectory ID 1014: Loss = 56.3124
Trajectory ID 1088: Loss = 56.3079
Trajectory ID 1011: Loss = 56.2068
Trajectory ID 5197: Loss = 56.1398
Trajectory ID 1006: Loss = 56.1246
Trajectory ID 2716: Loss 

In [90]:
# Get top trajectory ID (with highest loss)
top_traj_id = sorted_pairs[0][0]

# Retrieve from the original dataset
top_data = dataset[top_traj_id]
top_x, top_action_mask, top_timestep_mask = top_data

# Print nicely
print(f"\nTop Trajectory ID: {top_traj_id}")
print("Data (x):", top_x)




Top Trajectory ID: 1022
Data (x): tensor([[[ 1.1615,  0.6840,  0.1611,  ...,  0.4761,  0.5734, -1.2865],
         [ 0.0233, -0.4054, -0.8051,  ...,  0.4761,  0.5734, -1.2865],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0233, -0.4054, -0.8051,  ..., -0.0146, -0.4107,  1.2548],
         [ 1.1615,  0.6840,  0.1611,  ..., -0.0146, -0.4107,  1.2548],
         [ 0.9777, -0.7392, -0.4093,  ..., -0.0146, -0.4107,  1.2548],
         [-0.3114, -2.3895,  0.1585,  ..., -0.0146, -0.4107,  1.2548]],

        [[ 0.0233, -0.4054, -0.8051,  ...,  0.0656, -0.5021,  0.8280],
         [ 0.9777, -0.7392, -0.4093,  ...,  0.0656, -0.5021,  0.8280],
         [ 1.1615,  0.6840,  0.1611,  ...,  0.0656, -0.5021,  0.8280],
         [-0.3114, -2.3895,  0.1585,  ...,  0.0656, -0.5021,  0.8280]],

        ...,

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.